In [106]:
class Graph():
    def __init__(self):
        self.edges = {}     # keys are addresses, values are list with each address in list being an edge
        self.weights = {}   # dictionary keys are tuples with (address 1, address 2), value = weight between them
        self.n = 0          # number of vertices
        
    def add_edge(self,add1,add2,weight):
        try:
            self.edges[add1].append(add2)
        except:
            self.edges[add1] = []
            self.edges[add1].append(add2)
            self.n += 1
        try:
            self.edges[add2].append(add1)
        except:
            self.edges[add2] = []
            self.edges[add2].append(add1)

        self.weights[(add1,add2)] = weight
        self.weights[(add2,add1)] = weight
        
    def DFS_Main(self,start):
        vertices = [k for k in g.edges.keys()]
        color = {}
        for v in vertices:
            color[v] = "white"
        self.DFS(start,color)
    
    def DFS(self,v,colors):
        colors[v] = "grey"
        for w in self.edges[v]:
            if colors[w] == "white":
                self.DFS(w,colors)
        print(v,end=" ")
        colors[v] = "black"



In [107]:
import time
import blockcypher
import pandas as pd
import requests as r
import json


In [110]:
def analyze_address(search_add,g,depth,max_depth=5):
    time.sleep(0.2)
    address_details = blockcypher.get_address_details(search_add)
    for tx in address_details['txrefs']:
        tx_hash = tx["tx_hash"]
        is_input  = True if tx["tx_input_n"] == -1 else False
        if is_input and tx["tx_output_n"] == -1:
            print("concerning")
            break
        value = tx["value"] 
        tx_details = blockcypher.get_transaction_details(tx_hash)     
        if not is_input:
            receiving_add = ""
            if len(tx_details["outputs"]) == 1: # there is no change for the transaction
                receiving_add = tx_details["outputs"][0]["addresses"][0]
                weight = tx_details["outputs"][0]["value"]
            else: # there is change so we need to figure out which address is the recieving address and which is change
                if tx_details["outputs"][0]["value"] > tx_details["outputs"][1]["value"]:
                    receiving_add = tx_details["outputs"][0]["addresses"][0]
                    weight = tx_details["outputs"][0]["value"]
                else:
                    receiving_add = tx_details["outputs"][1]["addresses"][0]
                    weight = tx_details["outputs"][1]["value"]
            if receiving_add not in g.edges.keys():
                g.add_edge(search_add,receiving_add,value)
                if depth < max_depth:
                    g = analyze_address(receiving_add,g,depth + 1)
        
        else: #its an input to search address
            
            for inp in tx_details["inputs"]:
                output_add = inp["addresses"][0]
                value = inp["output_value"]
                if output_add not in g.edges.keys():
                    g.add_edge(output_add,search_add,value)
                    if depth < max_depth:
                        g = analyze_address(output_add,g,depth+1)
    
    return g
        
        
        
        
    

In [113]:
import blockcypher
search_add = "111K8kZAEnJg245r2cM6y9zgJGHZtJPy6"
g = Graph()
g = analyze_address(search_add,g,2)

    


RateLimitError: ('Status Code 429', '{"error": "Limits reached."}\n')

#### 

In [91]:
print(g.edges)

{'111K8kZAEnJg245r2cM6y9zgJGHZtJPy6': ['16USe1BH4d4pfHjrNqsHwzqa1bYWbkkRiy'], '16USe1BH4d4pfHjrNqsHwzqa1bYWbkkRiy': ['111K8kZAEnJg245r2cM6y9zgJGHZtJPy6', '1FsPmxTL2Yy4aWH5J7XYWWqTMhordXAoMn'], '1FsPmxTL2Yy4aWH5J7XYWWqTMhordXAoMn': ['16USe1BH4d4pfHjrNqsHwzqa1bYWbkkRiy', '1JHi3u3Dsq6vng8egJVKsFKXWhwzmcU3FB', '1DSpthr8whhgXfPAHDwq4M9JJnGSXXg1DZ', '14QuqBN9VFA8Csr4QLdJUCgaZT8WVsPgZe', '1FNGkomYw1Sae3L8r81MiwG9FH2vHFzGDG'], '1JHi3u3Dsq6vng8egJVKsFKXWhwzmcU3FB': ['1FsPmxTL2Yy4aWH5J7XYWWqTMhordXAoMn', '19iEzyHzqA6NfPN1WdESyMjrZXQzTm1Tef'], '19iEzyHzqA6NfPN1WdESyMjrZXQzTm1Tef': ['1JHi3u3Dsq6vng8egJVKsFKXWhwzmcU3FB', '1FCtXcAh5xGfQdU7jDw4q7yBnxeUyvBBVq'], '1FCtXcAh5xGfQdU7jDw4q7yBnxeUyvBBVq': ['19iEzyHzqA6NfPN1WdESyMjrZXQzTm1Tef'], '1DSpthr8whhgXfPAHDwq4M9JJnGSXXg1DZ': ['1FsPmxTL2Yy4aWH5J7XYWWqTMhordXAoMn', '1GNeV9gqLvGvZq6rg7KgTBYn1d44KxFEdR'], '1GNeV9gqLvGvZq6rg7KgTBYn1d44KxFEdR': ['1DSpthr8whhgXfPAHDwq4M9JJnGSXXg1DZ', '3J7JbQMDyL2aFNUwUdSh75QByGxTatL9PC', '3AQ2rgVX3wjKosoeVLLU3CYwVRJa59opcH'

In [92]:
g.DFS_Main(search_add)

1FCtXcAh5xGfQdU7jDw4q7yBnxeUyvBBVq 19iEzyHzqA6NfPN1WdESyMjrZXQzTm1Tef 1JHi3u3Dsq6vng8egJVKsFKXWhwzmcU3FB 3J7JbQMDyL2aFNUwUdSh75QByGxTatL9PC 3AQ2rgVX3wjKosoeVLLU3CYwVRJa59opcH 3FcatWJ6wtApcs6A6VRGBDFUhQoduXFhQS 3Kc7GakpQ1R2KQn87zMVqeCtyfp76pVxJJ 3NeEjN6tCCquY5webYbKGEVZYfvJas8p7Y 3QHoWCHWrHBPmpiGjwNpfZG1pUfpb45d2B 3D3eiUmP4E1x2GXBjiCFiKXUkJnWr9ULaf 36DNzLpNyGDCdsMrzBLHagyf45k1aMBakv 3MgAxSRWGyiJ9Wy7suML9uSFuuLFXKdHB2 3NRTzt49fQ6DYtyPnv21dU8Qgj9PmZGuco 1GNeV9gqLvGvZq6rg7KgTBYn1d44KxFEdR 1DSpthr8whhgXfPAHDwq4M9JJnGSXXg1DZ 1DYJ3Te3FCQNLYYnVkEj3JvZ15QNH27kog 1Ny3TJ5RNvZnKpw4HeH6JnG4t5ZPoqFuvw 19ftTwXT6kD1GdrcdyXootyoens6esNYZk 1HNG52hnozgNGHxXB8cmZdSNtM5QiDHtAB 139vN3ypkvLtd9y6dx8HM2D81v8Svo5Kv9 1Px49QxskiiwEhvQDnCtHCLUTFCkGUx62D 1PedEGzM9GyRfFaAAyp5VJycdpKbbL8w5p 1D4V2meoTEUucWrbRnGitV2pX3xEXJoeX7 13s5Z4gbCJ5tN4D5KkFRP5xv8Lyx7U1RVB 1FtV5SYN2Psvhv18gNRdAx6vcpYJzxiABc 16Aq7VgYZGoiXHGLayJtMSvVe7tJyWTGD7 1Bogsbfz3v5Rpwtr9H719VATMisfnb22xt 14QuqBN9VFA8Csr4QLdJUCgaZT8WVsPgZe 1AjtmvKRY2SLEStFQ5cp